In [10]:
# markdown settings
from IPython.display import display, HTML # type:ignore
style = '<style> body { font-family: ubuntu light; font-size: 16px; } </style>' 
display(HTML(style))

In [2]:
# major imports
import tensorflow as tf

In [3]:
# other imports
from tensorflow import keras
import matplotlib.pyplot as plt 
import pandas as pd

Data API $\rightarrow$ Helps in preprocessing of large dataset in Tensorflow. Create the dataset object and tell it where to get the data and how to transform it. TF cares all the details of preprocessing, multithreading, queueing, batching and prefetching. It also supports reading data from sql databases. 

TFRecord formt $\rightarrow$ A flexible and efficient binary format usually containing protocols buffers. 

To preprocess the data we can create custom layers dedicated for the the preprocessing of the data.

# <div style="font-family:fantasy;">The Data API</div>
The first dataset that we create is in the RAM itself not in the disk


In [4]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

we create a tensor <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 100">X</code> containing elements from 1 to 10 as numpy array 

by using <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">from_tensor_slices()</code> we create dataset from slices of <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 100">X</code>

Hence the dataset contains 10 items from 0 to 9
Alternatively we can use <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">tf.data.Dataset.range(10)</code>
then again for iteration on <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">datset</code> we can use 
```python
for item in dataset:
    print(item)
```

In [5]:
dataset = tf.data.Dataset.range(10)
for item in dataset: 
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
tf.Tensor(5, shape=(), dtype=int64)
tf.Tensor(6, shape=(), dtype=int64)
tf.Tensor(7, shape=(), dtype=int64)
tf.Tensor(8, shape=(), dtype=int64)
tf.Tensor(9, shape=(), dtype=int64)


# <div style="font-family:fantasy;">Chaining transformation</div>
* we perform transformations on the dataset via their methods each return the new transformed dataset.
* It helps in chaining transformations 


In [6]:
dataset = dataset.repeat(3).batch(7)

In [7]:
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)
tf.Tensor([8 9], shape=(2,), dtype=int64)


`repeat(3)`: creates 3 copies of the `dataset` and concatenates them to a single set and then another chained method is applied `batch()`\
`batch(7)`: creates batches of size 7 (7 elements) from the transformed `dataset` and returns it as new dataset. Now one problem being the end output being `tf.Tensor([8 9], shape=(2,), dtype=int64)` i.e 2 elements and not seven. We can drop it using `drop_remainder=True`

In [8]:
dataset = tf.data.Dataset.range(10).repeat(3).batch(7,drop_remainder=True)
for item in dataset: print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int64)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int64)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int64)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int64)


we can also use `map()` ofcourse in the `batch()` method as we do it for a list.

In [9]:
dataset = dataset.map(lambda x:x*2) #this will twice each element.
for item in dataset: print(item)

tf.Tensor([ 0  2  4  6  8 10 12], shape=(7,), dtype=int64)
tf.Tensor([14 16 18  0  2  4  6], shape=(7,), dtype=int64)
tf.Tensor([ 8 10 12 14 16 18  0], shape=(7,), dtype=int64)
tf.Tensor([ 2  4  6  8 10 12 14], shape=(7,), dtype=int64)


For intensive computations we might need multithreading we can do it by passing`num_parallel_threads`
<div style="border:2px solid;border-radius:4px;width:50vw;">The function passes to <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">map()</code> must be convertible t TF function we can use <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">apply()</code> method instead of <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">map()</code> that applies to tyhe dataset as whole rather than on single instances one by one.</div>

### <div style="font-family:fantasy;">shuffling the data</div>
Shuffling the instances in the dataset support generates more randomness thus being more helpful while while using gradinet descent.\
how it works ?
* it will first create a new dataset, that will start filling up a buffer with the first items of source dataset.
* Then whenever it is asked for an item, it will pull one out randomly from the buffer and replace it with a fresh one from the source dataset.
* This loops till the entire source dataset has been iterated.
* Now at this points it starts to pull data randomly from the buffer until it is empty.

We must specify the buffer sie and as it lies in the main memory we should take care that it does not create a stack overflow.

In [13]:
dataset = tf.data.Dataset.range(10).repeat(3)
dataset = dataset.shuffle(buffer_size=5,seed=42).batch(7) # seed is for random seed if we need same sequence of outputs everytime.
for items in dataset: print(items)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)


<div style="border:2px solid;border-radius:4px;width:50vw;">On calling <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">repeat()</code> on shuffled dataset by default it will generate a new order at every iteration. To stop this we can set <code style="font-size: medium;color:rgb(189, 255, 65);font-weight: 300">reshuffle_each_iteration=False</code></div>

For large dataset this method will not be efficient enough as the buffer size will be too small as compared to the dataset. The solution is to shuffle the source dataset itself.\
But as it is kept stagnant (like not shuffled during the training) so model might detect some spurious patterns and become prone to overfitting. Thus we split the dataset to multiple files and read them in random order during the training. Still instances located in same file may end up close to each other. To avoid this we can pick multiple files randomly, read them simultaneously by interleaving their records. Then on the top of it we can add a shuffling buffer using `shuffle()`.

First we load the california housing dataset and shuffle it and then we split it into training set, validation set and test set and then we split into many CSV files.

We keep store the location of training, validation and test set in `train_filepaths`, `valid_filepath`, `test_filepath` we create a dataset containing only these files paths.